In [11]:

import os 
import numpy as np
import geopandas as gpd
import pandas as pd
from datetime import datetime 


In [12]:


def datefix(datapath, out_name, reproject=True):

    ######## LOAD LWM SHAPEFILE ############
    lwm = lwm = gpd.read_file(datapath)
    lwm = lwm.assign(year=0)

    ########## LOOP THROUGH LWM DATES #########
    for i in range(len(lwm)):
        # print(f'\r{i} of {len(lwm)}', end='', flush=True) #print iteration
        lwm['Date_old'] = lwm['Date'] # make a copy of the 'old' date format 
        date = lwm.loc[i,'Date'] # subset date
        date.replace(" ", "") ### remove spaces
        date_len = len(date) # get length of date to help identify date format

        ### Replace / and - with underscore for consistent dates
        if '/' in date:
            date = date.replace("/", "_")
        elif '-' in date:
            date = date.replace("-", "_")

        date_str = date #initiate new date

        ##################### loop through different date formats #####################
        ##### Clean formats with YYYY or MM-DD-YYYY ######
        if date_len == 4: # if length is 4, contains only year
            date_str = date
        elif date_len == 10: # if length is 10, contains date as YYYY_MM_DD
            # check if format is YYYY_MM_DD or MM_DD_YYYY by trying to convert it to a date
            try:
                datetime.strptime(date, '%Y_%m_%d')
                date_str = date
            except:
                # flip it so YY-MM-DD
                month, day, year = date.split('_')
                date_str = "%s_%s_%s" %(year, month, day)
            
            
        #### Annas spot for manual fixes ####, can add another 'elif' to do more fixes:
        elif date == '04_102005':
            date_str = '2005_10_04'
            year = 2005

        ##### messier formats with missing leading zero or missing day: if len is between 4 and 10 ######
        elif date_len > 4:  ## if length is more than 4: 
            date_split = date # initiate split date

            # first look at how many date entries there are (is it m-yyyy or m-d-yyyy)
            #if string contains three entries: m-d-yyyy or m-dd-yyyy
            if len(date_split.split('_')) == 5: ## some datapoints have dublicate dates: mm-dd-yyyymm-dd-yyyy 
                month,day,yearbad,mbad,d_bad = date_split.split('_') 
                year=yearbad[0:4]
            if len(date_split.split('_')) == 3:
                month,day,year = date_split.split('_')
            #if string contains two entries there is no day: m-yyyy or mm-yyyy
            if len(date_split.split('_')) == 2:
                month, year = date_split.split('_')
                day = "01" # assume a day of 01, first of the month

            ### Add a leading zero to the month and date if they are only 1 character
            if len(month) == 1: # if month is missing leading zero
                month = "0%s" %month
            elif len(month) == 3: #some months have three digits, e.g. 005
                month = month[1:]

            if len(day) < 2: # if day is missing leading zero
                day = "0%s" %day 

            ## Construct new date_str with corrected year, month, day
            print(date_str, "was fixed to %s_%s_%s" %(year, month, day))
            date_str = "%s_%s_%s" %(year, month, day)
            # print(date_str, year)

        if len(date_str) > 10: ### for 1 date dublicate mm-dd-yyyymm-dd-yyyy
            date_str = date[0:10]
        #######  THE FINAL LWM WITH CORRECTED DATE #########
        lwm.loc[i,'Date'] = date_str
        lwm.loc[i,'year'] = date_str[:4]

        ####### Add decimal date 
        if len(date_str) == 10: # if the date format is normal 
            date_formated = datetime.strptime(date_str, '%Y_%m_%d') # format the date so python can read it
            decimal = int(datetime.strftime(date_formated,'%j')) / 365 # use %j to get day number of the year, (and make in into an integer (int)). divide by 365 to get fraction of the year
        elif len(date_str) == 4: #if the date only contains year, use 0.0 as decimal
            decimal = 0.0
        else: ### this is just a catch in case the dateformat is still wrong!!. 
            print('something is wroooong with this datem potentially make a manual fix', date_str)
        lwm.loc[i,'Date_dec'] = int(year) + round(decimal,2)


    #### uncomment below line if you want to delete the column 'Date_old'
    #lwm = lwm.drop(columns='Date_old')
    

    ####### REPROJECT LWM TO EPGS::32604 ##############
    LWM_32604 = lwm.to_crs('epsg:32604')
    if reproject:
        outpath = os.path.join(os.getcwd(), 'data', (out_name + '_32604.shp'))
        LWM_32604.to_file(outpath)
    else:
        #outpath = os.path.join(os.getcwd(), 'data', (out_name + '.shp'))
        lwm.to_file(out_name)

    return lwm


In [14]:
## Run the function: 

#### 'raw'/unfixed shapefile
LWM_path = "F:\Shoreline\Oahu_LWM_2023_update.shp"
#LWM_updated_name = 'Oahu_LWM_2023_date.shp'
# settigns 
LWM_path_datefix = "F:\Shoreline\Oahu_LWM_2023_update.shp"


# #### FIX DATES FOR SHAPEFILES 
LWMs = datefix(LWM_path, LWM_path_datefix, reproject=False)

LWMs

,Island,Area,year,Date,Uncert_m,Uncert_ft,Source,Date_old,geometry,Date_dec
0,Oahu,NanakuliKahe,2021,2021_05_19,0.103,0.337927,Honouliuli_TracksKahe_2021,2021_05_19,"LINESTRING Z (590130.908 2361838.017 0.000, 59...",2021.38
1,Oahu,Nanakuli,2021,2021_05_24,0.180,0.590551,Nanakuli_NanakuliBeach_2021,2021_05_24,"LINESTRING Z (589042.559 2363875.980 0.000, 58...",2021.39
2,Oahu,Nanakuli,2021,2021_05_24,0.081,0.265748,"Waianae_UlehawaBeachNorth_2021, Waianae_Ulehaw...",2021_05_24,"LINESTRING Z (588429.420 2364639.472 0.000, 58...",2021.39
3,Oahu,Makaha,2021,2021_05_24,0.088,2.887140,"Makaha_LahilahiNorth_2021, Makaha_MakahaBeachS...",2021_05_24,"LINESTRING Z (582325.689 2373136.102 0.000, 58...",2021.39
4,Oahu,Makaha,2021,2021_05_24,0.120,0.393701,"Makaha_MakahaBeachNorth_2021, Makaha_MakahaBea...",2021_05_24,"LINESTRING Z (580888.511 2374677.245 0.000, 58...",2021.39
...,...,...,...,...,...,...,...,...,...,...
2884,Oahu,Waik?k?,2022,2022_11_29,4.512,14.803000,HawaiiKaiWailupe2022mos.tif,2022_11_29,"LINESTRING Z (630444.111 2353645.488 0.000, 63...",2021.91
2885,Oahu,Waik?k?,2022,2022_11_29,4.512,14.803000,HawaiiKaiWailupe2022mos.tif,2022_11_29,"LINESTRING Z (629988.557 2353418.241 0.000, 62...",2021.91
2886,Oahu,Waik?k?,2022,2022_01_27,4.512,14.803000,Kahala2022mos.tif,2022_01_27,"LINESTRING Z (624893.491 2351096.581 0.000, 62...",2021.07
2887,Oahu,Waik?k?,2022,2022_01_27,4.512,14.803000,DiamondHead2022mos.tif,2022_01_27,"LINESTRING Z (624228.437 2350945.123 0.000, 62...",2021.07
